<a href="https://colab.research.google.com/github/kirtibala/Machine-learning/blob/main/Extraction_Name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",filename="mistral-7b-instruct-v0.1.Q8_0.gguf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.1.Q8_0.gguf:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q8_0.gguf'

In [7]:
import json
from pprint import pprint
from typing import List, Union

import requests
from llama_cpp import ChatCompletionMessage, Llama, LlamaGrammar

# Define the path to the model
MODEL_PATH = "/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q8_0.gguf"

# Initialize the Llama model
llm = Llama(model_path=MODEL_PATH)

# Load a grammar file
llama_grammar = LlamaGrammar.from_file("/content/json.gbnf")

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv  

In [5]:
!pip install llama_cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.8 MB/s eta 0:00:00
  Created wheel for llama_cpp-python: filename=llama_cpp_python-0.2.67-cp310-cp310-linux_x86_64.whl size=3560478 sha256=0d363a6513045aeac3bf2ac3b10d49066d2b47bdb3bc520801ad1342e12c0ee9
  Stored in directory: /root/.cache/pip/wheels/60/a9/cb/224bc1c1ff2c1bf04f50702ddb24c95a129263d48b042f001c
Successfully built llama_cpp-python


In [8]:
FUNCTIONS = [
    {
        "name": "Kirti_workfow",
        "description": "Exceute the workfow from the automationedge server",
        "parameters": {
            "type": "object",
            "properties": {
                "email_id": {
                    "type": "string",
                    "description": "email id of the person ",
                },
                "Mobile_No": {
                    "type": "number",
                    "description": "Mobile No of the person ",
                "Candidate_Name": {
                    "type": "string",
                    "description": "Name of Candidate"
                }
                },
            },
            "required": ["email_id","Mobile_No","Candidate_Name"],
        },
    }
]

In [9]:
# Generate chat sequence
def generate_chat_sequence(
    user_query: str, function_def: dict
) -> List[ChatCompletionMessage]:
    messages = [system_prompt]
    user_message = ChatCompletionMessage(role="user", content=user_query)
    function_message = ChatCompletionMessage(
        role="function", content=json.dumps(function_def)
    )
    messages.extend([user_message, function_message])
    return messages

In [10]:
# System prompt
system_prompt = ChatCompletionMessage(
    role="system",
    content="""Hello there! I'm here to assist you.
    I have the ability to extract the enitity for the functions to perform various tasks.
    Let's try something out.

""",
)

In [11]:
user_query="Hello,Everyone!! This is Kirtibala Gujar having email id xyz@gmail.com and mobile no is 8585856791"
messages = generate_chat_sequence(
   user_query , FUNCTIONS[0]
)

# Get response from the Llama model
response = llm.create_chat_completion(
    messages=messages, grammar=llama_grammar, temperature=0
)

# Extract assistant's content
assistant_content = response["choices"][0]["message"]["content"]


llama_print_timings:        load time =   26083.42 ms
llama_print_timings:      sample time =     334.69 ms /    46 runs   (    7.28 ms per token,   137.44 tokens per second)
llama_print_timings: prompt eval time =   26082.60 ms /   100 tokens (  260.83 ms per token,     3.83 tokens per second)
llama_print_timings:        eval time =   24430.13 ms /    45 runs   (  542.89 ms per token,     1.84 tokens per second)
llama_print_timings:       total time =   50999.90 ms /   145 tokens


In [12]:
assistant_content

'{\n"name": "Kirtibala Gujar",\n"email": "xyz@gmail.com",\n"phone_number": "8585856791"\n}'